### Librerias y datos

In [22]:
# Librerias
import pandas as pd
from sklearn.model_selection import train_test_split

# Libreria con las funciones personalizadas
from creditScoring_toolKit import *

In [7]:
# Cargamos datos
datos = pd.read_csv('Data/loan_data.csv')

### Base X_train, X_test, y_train, y_test

##### En caso de aun no haber creado las bases train y test

In [14]:
# Dividimos la base en train y test
train, test = train_test_split(datos, train_size=0.7,random_state=123)

In [15]:
# Definimos X_train, X_test, y_train, y_test

# Definimos X_train, X_test
X_train = train.drop(columns=['SK_ID_CURR','TARGET'])
X_test = test.drop(columns=['SK_ID_CURR','TARGET'])

# Definimos X_train, X_test
y_train = train['TARGET']
y_test = test['TARGET']

In [16]:
# Guardamos bases X_train, X_test, y_train, y_test
X_train.to_csv('Data/X_train.csv',index=False)
X_test.to_csv('Data/X_test.csv',index=False)
y_train.to_csv('Data/y_train.csv',index=False)
y_test.to_csv('Data/y_test.csv',index=False)

##### En caso de ya haber creado las bases train y test

In [17]:
# Cargamos datos de csv
X_train = pd.read_csv('Data/X_train.csv')
X_test = pd.read_csv('Data/X_test.csv')
y_train = pd.read_csv('Data/y_train.csv')
y_test = pd.read_csv('Data/y_test.csv')

### Análisis information value y categorias variables

In [20]:
# Clasificamos variables en categoricas y numericas
numericas, categoricas = var_numericas_categoricas(datos)

NameError: name 'var_numericas_categoricas' is not defined